In [5]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications import resnet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

import cv2
import matplotlib.pyplot as plt

In [6]:
for dirname, _, filenames in os.walk('./data'):
    print(dirname)

./data
./data/Uninfected
./data/Parasitized
./data/cell_images
./data/cell_images/Uninfected
./data/cell_images/Parasitized


In [7]:
def data_prep(parasitized, uninfected):
    
    parasitized = ["./data/Parasitized" + '/' +  parasitize for parasitize in parasitized]
    uninfected = ["./data/Uninfected" + '/' +  uninfect for uninfect in uninfected]

    labels = len(parasitized) * ['parasitized'] + len(uninfected) * ['uninfected']
    data = parasitized + uninfected

    return pd.DataFrame({'Image_Path': data , 'Labels': labels})

In [8]:
df = data_prep(os.listdir("./data/Parasitized/"), os.listdir("./data/Uninfected/"))

In [10]:
# Function to load and preprocess images
def load_images(file_paths, target_size=(64, 64)):
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        images.append(img)
    return np.array(images)

# Load and preprocess images
# Assuming df is your DataFrame with 'Image_Path' and 'Labels' columns
# Make sure to replace df with your actual DataFrame variable
X = load_images(df['Image_Path'])
y = df['Labels']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build an improved CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))  
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN model
model.fit(X_train, (y_train == 'parasitized').astype(int), epochs=30, batch_size=32, validation_split=0.2)

# Extract features from the CNN model
cnn_features_train = model.predict(X_train)
cnn_features_test = model.predict(X_test)

# Reshape features for Random Forest
cnn_features_train = cnn_features_train.reshape(cnn_features_train.shape[0], -1)
cnn_features_test = cnn_features_test.reshape(cnn_features_test.shape[0], -1)

# Build a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(cnn_features_train, (y_train == 'parasitized'))

# Predict using the hybrid model (Random Forest)
rf_predictions = rf_model.predict(cnn_features_test)

# Evaluate the hybrid model (Random Forest)
rf_accuracy = accuracy_score((y_test == 'parasitized'), rf_predictions)
print()
print(f"Accuracy of the hybrid model with Random Forest: {rf_accuracy*100}")

# Save models
if not os.path.exists('cnn-knn-models'):
    os.makedirs('cnn-rf-models')

# Save CNN model
rf_model.save('cnn-rf-models/cnn_model.h5')

# Save KNN model
import joblib
joblib.dump(rf_model, 'cnn-rn-models/knn_model.joblib')

Epoch 1/30
552/552 [==============================] - 84s 147ms/step - loss: 0.4059 - accuracy: 0.8272 - val_loss: 0.1958 - val_accuracy: 0.9338
Epoch 2/30
552/552 [==============================] - 78s 142ms/step - loss: 0.2001 - accuracy: 0.9358 - val_loss: 0.1485 - val_accuracy: 0.9546
Epoch 3/30
552/552 [==============================] - 79s 143ms/step - loss: 0.1734 - accuracy: 0.9461 - val_loss: 0.1396 - val_accuracy: 0.9537
Epoch 4/30
552/552 [==============================] - 81s 147ms/step - loss: 0.1483 - accuracy: 0.9516 - val_loss: 0.1419 - val_accuracy: 0.9578
Epoch 5/30
552/552 [==============================] - 79s 143ms/step - loss: 0.1382 - accuracy: 0.9555 - val_loss: 0.1245 - val_accuracy: 0.9578
Epoch 6/30
552/552 [==============================] - 80s 145ms/step - loss: 0.1269 - accuracy: 0.9598 - val_loss: 0.1402 - val_accuracy: 0.9599
Epoch 7/30
552/552 [==============================] - 81s 146ms/step - loss: 0.1242 - accuracy: 0.9588 - val_loss: 0.1501 - val_ac